In [47]:
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout




In [2]:
data=pd.read_csv(r'C:\Users\Adeel\Desktop\Deep Learning\Deep_learning\diabetes.csv')
data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
x=data.iloc[:,0:-1]
y=data.iloc[:,-1]


In [4]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)

In [5]:
scaler=StandardScaler()
x_train=scaler.fit_transform(x_train)
x_test=scaler.transform(x_test)

In [6]:
model=Sequential()
model.add(Dense(32,activation='relu',input_dim=8))
model.add(Dense(1,activation='linear'))

c:\Users\Adeel\Desktop\Deep Learning\Deep_learning\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [7]:
model.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [8]:
model.fit(x_train,y_train,epochs=100,batch_size=32,validation_data=(x_test,y_test))

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 5s 86ms/step - accuracy: 0.6315 - loss: 3.0451 - val_accuracy: 0.6948 - val_loss: 1.7861
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.6438 - loss: 2.9378 - val_accuracy: 0.7078 - val_loss: 1.5720
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.6269 - loss: 1.9517 - val_accuracy: 0.6818 - val_loss: 1.0751
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.6315 - loss: 1.4563 - val_accuracy: 0.6948 - val_loss: 1.1430
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.6888 - loss: 0.8492 - val_accuracy: 0.6948 - val_loss: 1.0629
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.6734 - loss: 0.8635 - val_accuracy: 0.7403 - val_loss: 1.0943
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.6658 - loss: 0.8695 - val_accuracy: 0.7468 - val_loss: 0.9357
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.6898 - loss: 0.8452 - val_accuracy: 0.

In [9]:
import kerastuner as kt

C:\Users\Adeel\AppData\Local\Temp\ipykernel_2360\1654478174.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


# How(tuning) to select the appropriate optmizer

In [10]:
def built_model(hp):
    model=Sequential()
    model.add(Dense(32,activation='relu',input_dim=8))
    model.add(Dense(1,activation='sigmoid'))
    optmizer=hp.Choice('optmizer',values=['adam','sgd','RMSprop','adadelta'])
    model.compile(optimizer=optmizer,loss='binary_crossentropy',metrics=['accuracy'])
    return model

In [11]:
tuner=kt.RandomSearch(built_model,objective='val_accuracy',max_trials=5)

Reloading Tuner from .\untitled_project\tuner0.json


In [12]:
tuner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

In [13]:
tuner.get_best_hyperparameters()[0].values

{'optmizer': 'RMSprop'}

In [14]:
model=tuner.get_best_models()[0]

c:\Users\Adeel\Desktop\Deep Learning\Deep_learning\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [15]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [16]:
model.fit(x_train,y_train,epochs=100,batch_size=32,validation_data=(x_test,y_test),initial_epoch=6)

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 49ms/step - accuracy: 0.7726 - loss: 0.5267 - val_accuracy: 0.7532 - val_loss: 0.4805
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7845 - loss: 0.5161 - val_accuracy: 0.7468 - val_loss: 0.4743
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7800 - loss: 0.4981 - val_accuracy: 0.7532 - val_loss: 0.4672
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7751 - loss: 0.4850 - val_accuracy: 0.7597 - val_loss: 0.4631
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7963 - loss: 0.4777 - val_accuracy: 0.7597 - val_loss: 0.4584
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7648 - loss: 0.4857 - val_accuracy: 0.7662 - val_loss: 0.4547
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7647 - loss: 0.4901 - val_accuracy: 0.7727 - val_loss: 0.4520
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7982 - loss: 0.4729 - val_accurac

# how to select no of nodes in initial layers

In [17]:
def built_model(hp):
    model=Sequential()
    model.add(Dense(hp.Int('units',min_value=8,max_value=128,step=8),activation='relu',input_dim=8))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='RMSprop',loss='binary_crossentropy',metrics=['accuracy'])
    return model

In [18]:



tuner = kt.RandomSearch(
    built_model,
    objective='val_accuracy',
    max_trials=5,
    directory='mydir',
    project_name='diabetes'
)

Reloading Tuner from mydir\diabetes\tuner0.json


In [19]:
x=tuner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))
print(x)

None


In [20]:
tuner.get_best_hyperparameters()[0].values

{'units': 64}

In [21]:
model=tuner.get_best_models()[0]

In [22]:
model.fit(x_train,y_train,epochs=100,batch_size=32,validation_data=(x_test,y_test),initial_epoch=6)

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 39ms/step - accuracy: 0.7496 - loss: 0.5475 - val_accuracy: 0.7987 - val_loss: 0.4858
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.7445 - loss: 0.5113 - val_accuracy: 0.8052 - val_loss: 0.4651
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7644 - loss: 0.5130 - val_accuracy: 0.8052 - val_loss: 0.4516
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.7813 - loss: 0.4682 - val_accuracy: 0.8052 - val_loss: 0.4460
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.7676 - loss: 0.4877 - val_accuracy: 0.8052 - val_loss: 0.4380
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7786 - loss: 0.4801 - val_accuracy: 0.8052 - val_loss: 0.4350
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.7798 - loss: 0.4819 - val_accuracy: 0.8117 - val_loss: 0.4305
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.7793 - loss: 0.4594 - val_accurac

# how to select no of layers

In [23]:
def built_model(hp):
    model=Sequential()
    model.add(Dense(64,activation='relu',input_dim=8))
    for i in range(hp.Int('num_layers',min_value=1,max_value=10)):
        model.add(Dense(64,activation='relu'))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='RMSprop',loss='binary_crossentropy',metrics=['accuracy'])
    return model


In [28]:
tuner=kt.RandomSearch(built_model,objective='val_accuracy',max_trials=3,directory='mydir',project_name='tuning layers')

c:\Users\Adeel\Desktop\Deep Learning\Deep_learning\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [29]:
tuner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

Trial 3 Complete [00h 00m 06s]
val_accuracy: 0.8116883039474487

Best val_accuracy So Far: 0.8116883039474487
Total elapsed time: 00h 00m 17s


In [30]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 4}

In [32]:
model=tuner.get_best_models(num_models=1)[0]

c:\Users\Adeel\Desktop\Deep Learning\Deep_learning\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\Adeel\Desktop\Deep Learning\Deep_learning\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [33]:
model.fit(x_train,y_train,epochs=100,batch_size=32,validation_data=(x_test,y_test),initial_epoch=6)

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 30ms/step - accuracy: 0.7862 - loss: 0.4395 - val_accuracy: 0.7922 - val_loss: 0.4270
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7650 - loss: 0.4715 - val_accuracy: 0.8117 - val_loss: 0.4242
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7912 - loss: 0.4379 - val_accuracy: 0.7792 - val_loss: 0.4505
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7926 - loss: 0.4268 - val_accuracy: 0.7922 - val_loss: 0.4300
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8326 - loss: 0.3999 - val_accuracy: 0.8117 - val_loss: 0.4320
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8138 - loss: 0.3923 - val_accuracy: 0.7922 - val_loss: 0.4452
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8007 - loss: 0.4202 - val_accuracy: 0.7727 - val_loss: 0.4715
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8322 - loss: 0.3775 - val_accurac

# now performing whole process

In [66]:
def built_model(hp):
    model=Sequential()
    counter=0
    for i in range(hp.Int('num_layers',1,10)):
        if counter==0:
            model.add(Dense(hp.Int('units'+str(i),min_value=8,max_value=64,step=4),
                        activation='relu',
                        input_dim=8))
            model.add(Dropout(hp.Float('dropout'+str(i),min_value=0.0,max_value=0.5,step=0.1)))
        else:

            model.add(Dense(hp.Int('units'+str(i),min_value=8,max_value=64,step=4),
                        activation=hp.Choice('activation'+str(i),values=['relu','tanh','sigmoid'])))
        counter+=1
    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='RMSprop',loss='binary_crossentropy',metrics=['accuracy'])
    return model


In [67]:
tuner=kt.RandomSearch(built_model,objective='val_accuracy',max_trials=3,directory='mydir',project_name='hyperpermeter tuning1')

c:\Users\Adeel\Desktop\Deep Learning\Deep_learning\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [68]:
tuner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

Trial 3 Complete [00h 00m 05s]
val_accuracy: 0.8051947951316833

Best val_accuracy So Far: 0.8051947951316833
Total elapsed time: 00h 00m 19s


In [51]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 3,
 'units0': 28,
 'units1': 36,
 'activation1': 'sigmoid',
 'units2': 48,
 'activation2': 'tanh',
 'units3': 64,
 'activation3': 'relu',
 'units4': 16,
 'activation4': 'relu'}

In [69]:
model=tuner.get_best_models(num_models=1)[0]

c:\Users\Adeel\Desktop\Deep Learning\Deep_learning\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [70]:
model.fit(x_train,y_train,epochs=100,batch_size=32,validation_data=(x_test,y_test),initial_epoch=6)

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 27ms/step - accuracy: 0.7595 - loss: 0.5105 - val_accuracy: 0.7662 - val_loss: 0.4282
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7762 - loss: 0.4764 - val_accuracy: 0.7532 - val_loss: 0.4235
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7714 - loss: 0.4769 - val_accuracy: 0.7792 - val_loss: 0.4251
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7658 - loss: 0.4659 - val_accuracy: 0.7727 - val_loss: 0.4352
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7496 - loss: 0.4905 - val_accuracy: 0.8052 - val_loss: 0.4214
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7586 - loss: 0.4852 - val_accuracy: 0.7922 - val_loss: 0.4267
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7351 - loss: 0.5072 - val_accuracy: 0.7987 - val_loss: 0.4249
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7899 - loss: 0.4737 - val_accuracy: 